## This short introduction uses Keras to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a Google Colaboratory notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select CONNECT.
2. Run all the notebook code cells: Select Runtime > Run all.
Download and install TensorFlow 2. Import TensorFlow into your program:

In [2]:
import tensorflow as tf
print(tf.__version__)

2.3.0


Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.05495679, -0.53369236, -0.24062102,  0.01995632, -0.07199247,
        -0.5356599 , -0.02008489,  0.36398387, -0.23084132,  0.03289154]],
      dtype=float32)

The tf.nn.softmax function converts these logits to "probabilities" for each class:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.11469281, 0.0636633 , 0.08534309, 0.11074795, 0.10101895,
        0.06353816, 0.10640109, 0.156223  , 0.08618182, 0.11218981]],
      dtype=float32)

The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.log(1/10) ~= 2.3.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.7561145

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss:

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 646us/step - loss: 0.2940 - accuracy: 0.9154
Epoch 2/5
1875/1875 [==============================] - 1s 623us/step - loss: 0.1411 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 1s 602us/step - loss: 0.1071 - accuracy: 0.9670
Epoch 4/5
1875/1875 [==============================] - 1s 546us/step - loss: 0.0871 - accuracy: 0.9736
Epoch 5/5
1875/1875 [==============================] - 1s 578us/step - loss: 0.0742 - accuracy: 0.9764


The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0763 - accuracy: 0.9777


[0.07634375244379044, 0.9776999950408936]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.61507489e-07, 1.96416083e-09, 4.01567959e-05, 2.35176296e-04,
        8.95157212e-12, 2.99477811e-08, 1.06960949e-14, 9.99723375e-01,
        4.82715130e-08, 9.57719976e-07],
       [2.98837257e-08, 7.31088403e-06, 9.99989271e-01, 3.41990540e-06,
        6.99486909e-13, 3.23088045e-08, 4.54127083e-08, 3.05922036e-13,
        4.13513455e-08, 1.34895788e-15],
       [5.28303815e-07, 9.99542236e-01, 1.33033987e-04, 2.93716471e-06,
        2.10695998e-05, 2.06420664e-05, 8.38631240e-06, 2.53929233e-04,
        1.68202714e-05, 5.04467437e-07],
       [9.99989152e-01, 7.95646101e-12, 3.68573319e-06, 4.74755133e-08,
        1.85391293e-08, 2.10023177e-06, 1.81992334e-06, 2.10057033e-06,
        1.00926885e-08, 1.09262953e-06],
       [2.48017784e-07, 7.02720954e-12, 3.05628964e-06, 1.93587137e-08,
        9.98697221e-01, 6.44153317e-07, 4.90760385e-06, 6.88069633e-07,
        4.76615554e-07, 1.29285944e-03]], dtype=float32)>